## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

model_path='My Drive/AI For Good - AI Blitz 3/AutoDrive/Models/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
import requests
import cv2
import shutil
import glob

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# from sklearn.metrics import roc_auc_score, roc_curve, auc
# from sklearn.model_selection import train_test_split

# Load the TensorBoard notebook extension
%load_ext tensorboard

from google.colab.patches import cv2_imshow 

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

# Data Loading

In [ ]:
# custom copytree because Colab doesn't have the latest version of shutil.copytree which now contains the dirs_exist_ok flag solving this issue
# copied verbatim from https://stackoverflow.com/a/12514470/5991868
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
# home_dir is the home directory for the images, from which training and validation splits will be taken
home_dir='/content/All_Data/'

# this can take a bit (maybe 30s)
if not os.path.exists(home_dir):
  os.mkdir(home_dir)
  for cview in ['Front','Left','Rear','Right']:
    os.mkdir(os.path.join(home_dir,'camera'+cview))

  urlbase = 'https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/autodri/v0.1/'

  for filename in ['test']:
    # create a temporary directory using TemporaryDirectory and context manager and unzip to there
    with TemporaryDirectory() as tmpdirname:
      # download the file
      requrl = requests.get(f'{urlbase}{filename}.zip')
      with open(f'{tmpdirname}/{filename}.zip', "wb") as zip:
        zip.write(requrl.content)
      # unzip
      with ZipFile(f'{tmpdirname}/{filename}.zip','r') as zip_ref:
        zip_ref.extractall(tmpdirname)
      # copy the data to the All_Data dir
      shutil.copy2(f'{tmpdirname}/{filename}/{filename}.csv',f'{home_dir}/{filename}.csv')
      for cview in ['Front','Left','Rear','Right']:
        copytree(f'{tmpdirname}/{filename}/camera{cview}/',f'{home_dir}/camera{cview}/')


In [ ]:
combdf=pd.read_csv(f'{home_dir}/test.csv')

# creat the img_list column
combdf['img_list']=combdf['filename'].apply(
    lambda fname: [os.path.join(home_dir,f'camera{cview}/{fname}') for cview in ['Left','Front','Right','Rear']])

In [ ]:
combdf.head()

,filename,img_list
0,0.jpg,"[/content/All_Data/cameraLeft/0.jpg, /content/..."
1,1.jpg,"[/content/All_Data/cameraLeft/1.jpg, /content/..."
2,2.jpg,"[/content/All_Data/cameraLeft/2.jpg, /content/..."
3,3.jpg,"[/content/All_Data/cameraLeft/3.jpg, /content/..."
4,4.jpg,"[/content/All_Data/cameraLeft/4.jpg, /content/..."


# Model Setup

In [ ]:
def get_imgs(imlist):
  imgs=[]
  for impath in imlist:
    img = load_img(impath,target_size=(224, 224))
    imgs.append(img_to_array(img))
    # Pillow images should be closed after `load_img`,
    # but not PIL images.
    if hasattr(img, 'close'):
                img.close()
                
  return np.stack(imgs,axis=0)

def imageseq_generator(df, batch_size = 64):  
  inds=df.index.to_list()
  # while True:
    # shuffle the indices for the epoch
    # np.random.shuffle(inds)

  # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
  for offset in range(0, len(inds), batch_size):
    # Get the samples you'll use in this batch
    batch_inds = inds[offset:(offset+batch_size)]

    batch_input  = []
    batch_output = []     
    # Read in each input, perform preprocessing and get labels
    for ind in batch_inds:
        batch_input.append(get_imgs(df.loc[ind,'img_list']))
        # batch_output.append(df.loc[ind,'canSteering'])

    # Return a tuple of (input, output) to feed the network    
    # yield (np.array(batch_input), np.array(batch_output))
    yield np.array(batch_input)

In [ ]:
# model = keras.models.load_model(f'{os.path.join(mount_path,model_path)}200903_1959_cv0_AutoDrive_frozen');
# preds=model.predict(imageseq_generator(combdf, batch_size = 64),verbose=1)

In [ ]:
# set tensorflow verbosity to error so that endless warnings aren't printed to the screen upon model_load
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model_files=os.listdir(f'{os.path.join(mount_path,model_path)}')

# do the big models first to verify that the memory is set up ok
model_files.reverse()

all_preds=np.empty((combdf.shape[0],len(model_files)))

for model_num, model_file in enumerate(model_files):
  print(model_file)

  model = keras.models.load_model(f'{os.path.join(mount_path,model_path)}{model_file}')

  all_preds[:,model_num] = np.squeeze(model.predict(imageseq_generator(combdf, batch_size = 32), verbose=1))


200904_2345_DenseNet_cv1_AutoDrive_frozen
665/665 [==============================] - 197s 296ms/step
200904_2030_DenseNet_cv0_AutoDrive_frozen
665/665 [==============================] - 194s 292ms/step
200904_1734_DenseNet_cv2_AutoDrive_frozen
665/665 [==============================] - 195s 293ms/step
200904_1542_Xception_cv2_AutoDrive_frozen
665/665 [==============================] - 195s 293ms/step
200904_1325_Xception_cv1_AutoDrive_frozen
665/665 [==============================] - 195s 293ms/step
200904_1020_Xception_cv0_AutoDrive_frozen
665/665 [==============================] - 195s 293ms/step
200904_0909_cv2_AutoDrive_frozen
665/665 [==============================] - 218s 328ms/step
200904_0725_cv1_AutoDrive_frozen
665/665 [==============================] - 218s 328ms/step
200903_1959_cv0_AutoDrive_frozen
665/665 [==============================] - 218s 328ms/step


In [ ]:
np.save(mount_path+'My Drive/AI For Good - AI Blitz 3/AutoDrive/Results/all_preds.npy',all_preds)

In [ ]:
predmed=np.mean(all_preds,axis=1)

In [ ]:
predmed.shape

(21269,)

In [ ]:
submitdf=pd.DataFrame(data=predmed,columns=['canSteering'])
submitdf['filename']=combdf['filename']
submitdf.head()

,canSteering,filename
0,-20.094024,0.jpg
1,0.848319,1.jpg
2,-5.274710,2.jpg
3,-28.838352,3.jpg
4,-55.952873,4.jpg


In [ ]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
submitdf.to_csv(home_dir+datestr+'_submission.csv',index=True)

In [ ]:
for model_num, model_file in enumerate(model_files):
  submitdf=pd.DataFrame(data=all_preds[:,model_num],columns=['canSteering'])
  submitdf['filename']=combdf['filename']
  submitdf.head()

  datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
  submitdf.to_csv(home_dir+datestr+'_sub_'+model_file+'.csv',index=True)

In [ ]:
len(model_files)

9